# Setup 🏗️


In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler




In [2]:
viscos = 1/5200

# load RANS data created by rans.m (which can be downloaded)
# load DNS data
DNS_mean  = np.genfromtxt("LM_Channel_5200_mean_prof.dat",comments="%").transpose()
y_DNS     = DNS_mean[0]
yplus_DNS = DNS_mean[1]
u_DNS     = DNS_mean[2]
dudy_DNS  = np.gradient(u_DNS,y_DNS)

DNS_stress = np.genfromtxt("LM_Channel_5200_vel_fluc_prof.dat",comments="%").transpose()

uu_DNS = DNS_stress[2]
vv_DNS = DNS_stress[3]
ww_DNS = DNS_stress[4]
uv_DNS = DNS_stress[5]
uw_DNS = DNS_stress[6]
vw_DNS = DNS_stress[7]
k_DNS  = 0.5*(uu_DNS+vv_DNS+ww_DNS)

DNS_RSTE = np.genfromtxt("LM_Channel_5200_RSTE_k_prof.dat",comments="%")

eps_DNS = DNS_RSTE[:,7]/viscos # it is scaled with ustar**4/viscos

# fix wall
eps_DNS[0]=eps_DNS[1]

# load data from k-omega RANS
data  = np.loadtxt('y_u_k_om_uv_5200-RANS-code.txt').transpose()
y     = data[0]
u     = data[1]
k     = data[2]
om    = data[3]
diss1 = 0.09*k*om
ustar = (viscos*u[0]/y[0])**0.5
yplus = y*ustar/viscos

# dont train on, uu, vv, ww, uv, uw, vw 
# Maybe mixed terms are ok (just not uu,vv,ww)

#-----------------Data_manipulation--------------------

# Delete first value for all interesting data
uv_DNS    = np.delete(uv_DNS, 0)
vv_DNS    = np.delete(vv_DNS, 0)
ww_DNS    = np.delete(ww_DNS, 0)
uw_DNS    = np.delete(uw_DNS,0)
vw_DNS    = np.delete(vw_DNS,0)
k_DNS     = np.delete(k_DNS, 0)
eps_DNS   = np.delete(eps_DNS, 0)
dudy_DNS  = np.delete(dudy_DNS, 0)
yplus_DNS = np.delete(yplus_DNS,0)
uu_DNS    = np.delete(uu_DNS,0)
y_DNS     = np.delete(y_DNS,0)
u_DNS     = np.delete(u_DNS,0)

# Calculate ny_t and time-scale tau
viscous_t = k_DNS**2/eps_DNS 
tau       = viscous_t/abs(uv_DNS)

# Calculate c_1, c_2, & c_3 of the Non-linear Eddy Viscosity Model
# Array for storing c_1, c_2, & c_3
c_0 = -2*(ww_DNS/k_DNS - 2/3)/(tau**2*dudy_DNS**2)
c_2 = 2*((ww_DNS/k_DNS - 2/3) + (uu_DNS/k_DNS - 2/3))/(tau**2*dudy_DNS**2)

c = np.array([c_0,c_2])

dudy_squared_DNS = (dudy_DNS**2).reshape(-1,1)

#TODO ML using PyTorch to estimate c_1, c_2, & c_3


Setting up input and output as tensors 

In [3]:

X = StandardScaler().fit_transform(dudy_squared_DNS)

# transpose the target vector to make it a column vector
y = c.transpose()

# split the feature matrix and target vector into training and validation sets
# test_size=0.2 means we reserve 20% of the data for validation
# random_state=42 is a fixed seed for the random number generator, ensuring reproducibility
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# convert the numpy arrays to PyTorch tensors with float32 data type
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# create PyTorch datasets and dataloaders for the training and validation sets
# a TensorDataset wraps the feature and target tensors into a single dataset
# a DataLoader loads the data in batches and shuffles the batches if shuffle=True
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Let's set up a neural network:

In [4]:
class ThePredictionMachine(nn.Module):

    def __init__(self):
        
        super(ThePredictionMachine, self).__init__()

        self.input   = nn.Linear(1, 50)     
        self.hidden1 = nn.Linear(50, 25)    
        self.hidden2 = nn.Linear(25, 2)     

    def forward(self, x):
        x = nn.functional.relu(self.input(x))
        x = nn.functional.relu(self.hidden1(x))
        x = self.hidden2(x)
        return x


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches

    print(f"Avg loss: {test_loss:>8f} \n")

In [6]:
# Instantiate a neural network
neural_net = ThePredictionMachine()

# Set up hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 1000

# Initialize the loss function
loss_fn = nn.MSELoss()

# Choose loss function, check out https://pytorch.org/docs/stable/optim.html for more info
# In this case we choose Stocastic Gradient Descent
optimizer = torch.optim.SGD(neural_net.parameters(), lr=learning_rate)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, neural_net, loss_fn, optimizer)
    test_loop(val_loader, neural_net, loss_fn)
print("Done!")


Epoch 1
-------------------------------
Avg loss: 0.009120 

Epoch 2
-------------------------------
Avg loss: 0.008328 

Epoch 3
-------------------------------
Avg loss: 0.007640 

Epoch 4
-------------------------------
Avg loss: 0.007040 

Epoch 5
-------------------------------
Avg loss: 0.006370 

Epoch 6
-------------------------------
Avg loss: 0.005913 

Epoch 7
-------------------------------
Avg loss: 0.005508 

Epoch 8
-------------------------------
Avg loss: 0.005075 

Epoch 9
-------------------------------
Avg loss: 0.004757 

Epoch 10
-------------------------------
Avg loss: 0.004471 

Epoch 11
-------------------------------
Avg loss: 0.004214 

Epoch 12
-------------------------------
Avg loss: 0.003980 

Epoch 13
-------------------------------
Avg loss: 0.003767 

Epoch 14
-------------------------------
Avg loss: 0.003572 

Epoch 15
-------------------------------
Avg loss: 0.003393 

Epoch 16
-------------------------------
Avg loss: 0.003227 

Epoch 17
--------

In [7]:
preds = neural_net(X_val_tensor)

In [8]:
print(preds)


tensor([[ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0203, -0.0013],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0008,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0008,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0011,  0.0024],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0007,  0.0022],
        [ 0.0008,  0.0022],
        [ 0.0015,  0.0025],
        [ 0.0008,  0.0022],
        [ 0.0066,  0

In [9]:
print(y_val_tensor)

tensor([[1.7618e-03, 2.3648e-03],
        [1.1746e-03, 2.1416e-03],
        [7.4928e-04, 1.5384e-03],
        [1.3403e-03, 2.4060e-03],
        [1.3561e-03, 2.4290e-03],
        [1.9260e-03, 2.1569e-03],
        [1.3856e-03, 2.4373e-03],
        [7.8223e-04, 1.5823e-03],
        [1.1771e-03, 2.1452e-03],
        [1.3981e-03, 2.4493e-03],
        [9.2183e-05, 2.0254e-04],
        [1.4739e-03, 2.3934e-03],
        [1.8799e-03, 2.2546e-03],
        [1.3216e-03, 2.3735e-03],
        [1.4120e-03, 2.3917e-03],
        [6.2119e-04, 1.3829e-03],
        [7.1264e-04, 1.4908e-03],
        [9.4952e-04, 1.8125e-03],
        [1.9794e-03, 2.1092e-03],
        [5.2573e-04, 1.3203e-03],
        [1.2978e-03, 2.3347e-03],
        [1.4405e-03, 2.3849e-03],
        [1.5124e-03, 2.3909e-03],
        [1.1319e-03, 2.0767e-03],
        [4.2858e-04, 1.4473e-03],
        [7.8594e-04, 1.5873e-03],
        [1.9138e-03, 2.1842e-03],
        [1.9107e-03, 2.2182e-03],
        [1.3490e-03, 2.4201e-03],
        [1.604